
### 1. Having Access to `MDLS_API_KEY` from `.env` File

With this approach, you avoid writing an API key in the code. Instead, you put it in the `.env` file. The code below reads the variables in the `.env` file and sets them as environment variables.
In this particular case, we only defined one variable: `MDLS_API_KEY`.

**Important**: put the `.env` file in the same location as this file, or the file that will call the LLM. For example, if the LLM is going to be called from an AHK file, then place the `.env` file in the same location as the AHK file.

The contents of the `.env` file should look like this:
```plaintext
Key_1=Value_1
Key_2=Value_2
...
Key_n=Value_n
```
**To check use:** 
```plaintext
    1. print(os.environ.get("MDLS_API_KEY")), or 
    2. print(os.environ["MDLS_API_KEY"])
```

In [1]:
import os

# Read variables from .env file and sets them as environment variables.
with open(".env") as f:
    for line in f:
        key, value = line.strip().split("=")
        os.environ[key] = value

# Get the environment var MDLS_API_KEY and use it in a print statement to check. 
# You can use it for anything you want, including to configure and use a LLM model :)
# print(f'MDLS_API_KEY is: {os.environ["MDLS_API_KEY"]}')

### 2. Instantiates the model to generate Text from Text

In [ ]:
import google.generativeai as genai 

genai.configure(api_key=os.environ["MDLS_API_KEY"])
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
response = model.generate_content("Cuanto es la suma de 6 y 6?")
print(response.text)

### 3. Instantiates the model to generate Text from Text & local Image   
###### ***If using a single image, place the text prompt after the image.***

In [ ]:
import google.generativeai as genai #
import PIL.Image

genai.configure(api_key=os.environ["MDLS_API_KEY"]) #
model = genai.GenerativeModel(model_name="gemini-1.5-flash") #
organ = PIL.Image.open("media/organ.jpg")
prompt = "Tell me about this instrument"
response = model.generate_content([organ, prompt])
print(response.text)

### 4. Instantiates the model to initiate a Chat using only Text input

In [ ]:
import google.generativeai as genai #

genai.configure(api_key=os.environ["MDLS_API_KEY"]) #
model = genai.GenerativeModel(model_name="gemini-1.5-flash") #
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)
response = chat.send_message("I have 2 dogs in my house.")
print(response.text)
response = chat.send_message("How many paws are in my house?")
print(response.text)

### 5. Instantiates the model configuring **max_output_tokens** and **temperature** to generate content using only Text input   
###### ***It's also possible to set config options independently in each generate_content request***

In [ ]:
import google.generativeai as genai #

genai.configure(api_key=os.environ["MDLS_API_KEY"]) #
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", 
    generation_config=genai.types.GenerationConfig(max_output_tokens=10, temperature=0.0))

response = model.generate_content("Tell me how many stars has the United States flag.")
print(response.text)

response = model.generate_content("Tell me how many stars has the United States flag.")
print(response.text)


### 6. Instantiates the model configuring **temperature** and **system_instruction** to generate content using only Text input   
###### ***It's also possible to set config options independently in each generate_content request***

In [ ]:
import google.generativeai as genai #

genai.configure(api_key=os.environ["MDLS_API_KEY"]) #
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", 
    generation_config=genai.types.GenerationConfig(temperature=0.1), 
    system_instruction=
    """
    You are an expert coder specialized in Python with more than 10 years of experience. You will help me with any python code that I need. Although you can read Spanish fluently, I may ask in spanish, but you will always answer in english.
    Algunas veces querré que me des código para resolver una tarea puntual sin darte mayor detalle; en estos casos, usarás tu experiencia para interpretar mi requerimiento. Otras veces te daré la estructura de una función y su documentación, para que tu generes lo que falta para completar la función.
     
    Por ejemplo, si te paso:
    def factorial(n:int) -> int:
        '''
        Calcula el factorial de un número entero no negativo usando recursividad.

        Args:
            n (int): Número entero no negativo para calcular el factorial.

        Returns:
            int: Factorial de n.

        Raises:
            ValueError: Si n es un número negativo.
        '''

    Tu respuesta será:
        if n < 0:
            raise ValueError("The Factorial is not defined for negative numbers.")
        elif n == 0:
            return 1
        else:
            return n*factorial(n-1)

    IMPORTANTE: Quiero el código sin comentarios.
    """
    )

response = model.generate_content(
    """
    def fibonacci(n:int) -> int:
        '''
        Devuelve el n-ésimo número de la serie de fibonacci.

        Args:
            n (int): Número entero no negativo.

        Returns:
            int: el n-ésimo número de la serie de fibonacci.

        Raises:
            ValueError: Si n es un número negativo.
        '''
    """
    )
print(response.text)

### 7. Upload two PDF files to the cloud* so the LLM model can have access to them. The process finalize deleting the uploaded files   
###### *The File API, where you have up to 20 GB space updated every 48 hours. Files can be up to 2 GB each.

In [ ]:
import google.generativeai as genai #

genai.configure(api_key=os.environ["MDLS_API_KEY"]) #

# Upload as many files as you need and be consistent below in response = model.generate_...
myfile1 = genai.upload_file("media/PoderMAZG_Firmado.pdf")
myfile2 = genai.upload_file("media/PoderMAZG_Firmado2.pdf")

prompt = "Summarize the differences between these documents."

model = genai.GenerativeModel(model_name="gemini-1.5-flash") #
response = model.generate_content([prompt, myfile1, myfile2])
print(response.text)

# Delete the documents from the File API
for f in genai.list_files():
    f.delete()
    print(f.name, " - ", f.mime_type, " deleted.")
    

### 8. Instantiates the model, setting streaming response, for real-time content generation using only Text input

This approach allows you to get responses as they are being generated, which is useful for creating more interactive applications.


In [ ]:
import google.generativeai as genai

genai.configure(api_key=os.environ["MDLS_API_KEY"])
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

prompt = "Write a story about a space adventure"
response = model.generate_content(prompt, stream=True)

for chunk in response:
    print(chunk.text, end="")


### 9. Instantiates the model using safety settings to generate filtered content using only Text input

This approach allows you to customize content filtering thresholds for generated responses.

In [ ]:
import google.generativeai as genai

genai.configure(api_key=os.environ["MDLS_API_KEY"])

safety_settings = {
    "harassment": "BLOCK_MEDIUM_AND_ABOVE",
    "hate_speech": "BLOCK_MEDIUM_AND_ABOVE",
    "sexually_explicit": "BLOCK_MEDIUM_AND_ABOVE",
    "dangerous": "BLOCK_MEDIUM_AND_ABOVE"
}

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    safety_settings=safety_settings
)

response = model.generate_content("Tell me the movies of Ilona Staller with a short description")
print(response.text)

### 10. Instantiates the model to generate Text from Text & web Image   
###### ***If using a single image, place the text prompt after the image.***

In [6]:
import google.generativeai as genai #
import httpx
import os
import base64

genai.configure(api_key=os.environ["MDLS_API_KEY"]) #
model = genai.GenerativeModel(model_name="gemini-1.5-flash") #

image1_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg/2560px-Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg"
image1 = {'mime_type':'image/jpeg', 'data': base64.b64encode(httpx.get(image1_path).content).decode('utf-8')}

prompt = "Caption this image."

response = model.generate_content([image1, prompt])

print(response.text)

Here is a caption for the image:

A panoramic view of London, England on an overcast day. The iconic landmarks of the London Eye, Big Ben, and the Shard are prominently featured, surrounded by a mix of historical and modern architecture. Lush green trees add a touch of nature to the urban landscape.
